In [1]:
from datetime import datetime
import json

In [2]:
import anvil.server

anvil.server.connect("client_KYOM4YFDIE4TMEO3UAOTEPRC-55C7JZ62MGB2UXA6")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Published" as CLIENT


In [3]:
from anvil.tables import app_tables

questions = app_tables.questions.search()

In [7]:
from dotenv import load_dotenv
import json_repair
import os

import google.generativeai as genai


key_gemini = os.environ["GOOGLE_API_KEY"]

genai.configure(api_key=key_gemini)

In [8]:
# Create the model
generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": 8192,
    "response_mime_type": "application/json",
}

llm_gemini = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config=generation_config,
    # safety_settings = Adjust safety settings
    # See https://ai.google.dev/gemini-api/docs/safety-settings
)

In [10]:
json_schema = """
  {
    "topic_description": {
      "type": "string",
      "description": "A sentence describing the sub-topic to which the question belongs. That means this sentence should specify in a granular level what specific sub-topic the question belongs to. It should be abstract in a way that other questions could be put in this description too. Use between 5 and 10 words."
    },
    "level": {
      "type": "string",
      "description": "The difficulty level of the question. It should be only one of the following options: 'beginner', 'intermediate', 'advanced'."
    },
    "question": {
      "type": "string",
      "description": "The actual question text. It should be a question of type TRUE or FALSE. It means that the questions should be an assertion that could be answered with TRUE or FALSE."
    },
    "answer_correct": {
      "type": "string",
      "description": "The correct answer to the question. It should be only one of the following options: TRUE or FALSE"
    },
    "explanation": {
      "type": "string",
      "description": "An explanation or solution to the question."
    }
  }
"""

In [11]:
prompt_question_generator = """
    TASK CONTEXT:
    I am studying DATASTRUCTURES AND ALGORITHMS and I need to practice some questions on this topic. 
    
    The objective is to memorize some patterns.
    
    TASK DESCRIPTION:
    I would like you to generate a list of TRUE or FALSE questions.
    Each question should be in the form of a statement that could be either TRUE or FALSE.
    I will provide an example code in the CODE section.
    You should not change the code.
    You should not output the code, because it will be showed to the user automaticaly.
    I want questions that ask what the code do. I some questions you should assert something false, in other something true about the code.
    You should think in what this code could be used in a real case, or in DSA questions.

    
    CODE:
    {code}
    
    TASK REQUIREMENTS:
    Please refrain from creating questions that require mathematical calculations, but you may create questions with mathematical formulas.
    You SHOULD use LATEX to write mathematical formulas and code, but you should use the Katex flavor.
    Also you should put $$ in the beggining of the katex code and $$ at the end of the code. This is necessary because the interpreter needs it.
    
    TASK DETAILS:
    You should create {quantity} questions of level {level}.
   
    
    FORMAT OUTPUT INSTRUCTIONS:
    It should be formatted in list of JSON objects as described below.
    {json_schema}
"""

In [12]:
code1 = """
        x = int(input())
        
        a = 10
        b = 5
        
        bigger = max(a, b)
        minor = min(a, b)
        
        if x % bigger != 0 and x % minor != 0:
            print(-1)
        else:
            print( x // bigger + x % bigger // minor )
    """

In [19]:
parameters = {
    "title": "DSA - Code questions",
    "quantity": "5",
    "level": "medium",
    "code": code1,
}

In [14]:
prompt_question_generator_formatted = prompt_question_generator.format(
    code=parameters["code"],
    quantity=parameters["quantity"],
    level=parameters["level"],
    json_schema=json_schema,
)

response = llm_gemini.generate_content(prompt_question_generator_formatted)
questions = json_repair.loads(response.text)

In [21]:
if not isinstance(questions, list):
        questions = [questions]

for question in questions:
    print(json.dumps(question, indent=4))
    # add to the database
    app_tables.questions.add_row(
        created_at=datetime.now(),
        title=parameters["title"],
        topic_description=question["topic_description"],
        level=question["level"],
        question=question["question"],
        type="true_or_false",
        answer_correct=f"{question['question']} \n\n {code1}",
        answers=None,
        explanation=question["explanation"],
        # user=anvil.users.get_user(),
    )

{
    "topic_description": "Modulo operation and conditional statements",
    "level": "intermediate",
    "question": "The code calculates the sum of two divisions:  the integer division of the input number by the bigger number and the integer division of the remainder of the first division by the smaller number.",
    "answer_correct": "TRUE",
    "explanation": "The code first checks if the input number is divisible by either the bigger or smaller number. If not, it prints -1. Otherwise, it calculates the integer division of the input number by the bigger number, and then calculates the integer division of the remainder of the first division by the smaller number. Finally, it prints the sum of these two results."
}
{
    "topic_description": "Modulo operation and conditional statements",
    "level": "intermediate",
    "question": "The code will always return a negative number if the input number is not divisible by either $\\frac{a}{b}$ or $\\frac{b}{a}$.",
    "answer_correct": "